# Ajuste Fino de Modelos da Open AI

Este notebook é baseado nas orientações atuais fornecidas na documentação de [Ajuste Fino](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) da Open AI.

O ajuste fino melhora o desempenho dos modelos de base para sua aplicação ao re-treiná-los com dados e contexto adicionais relevantes para aquele caso de uso ou cenário específico. Vale lembrar que técnicas de engenharia de prompts como _few shot learning_ e _retrieval augmented generation_ permitem aprimorar o prompt padrão com dados relevantes para melhorar a qualidade. No entanto, essas abordagens são limitadas pelo tamanho máximo da janela de tokens do modelo de base escolhido.

Com o ajuste fino, estamos efetivamente re-treinando o próprio modelo com os dados necessários (permitindo usar muito mais exemplos do que caberiam na janela máxima de tokens) – e implantando uma versão _personalizada_ do modelo que não precisa mais receber exemplos no momento da inferência. Isso não só melhora a eficácia do nosso design de prompt (temos mais flexibilidade para usar a janela de tokens para outras finalidades), como também pode reduzir nossos custos (diminuindo a quantidade de tokens que precisamos enviar ao modelo durante a inferência).

O ajuste fino tem 4 etapas:
1. Preparar os dados de treinamento e fazer o upload.
1. Executar o trabalho de treinamento para obter um modelo ajustado.
1. Avaliar o modelo ajustado e iterar para melhorar a qualidade.
1. Implantar o modelo ajustado para inferência quando estiver satisfeito.

Vale lembrar que nem todos os modelos de base suportam ajuste fino – [consulte a documentação da OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) para as informações mais recentes. Você também pode ajustar novamente um modelo que já foi ajustado anteriormente. Neste tutorial, vamos usar o `gpt-35-turbo` como nosso modelo de base para ajuste fino.

---


### Etapa 1.1: Prepare seu Conjunto de Dados

Vamos criar um chatbot que te ajuda a entender a tabela periódica dos elementos respondendo perguntas sobre um elemento com uma limerique. Neste tutorial simples, vamos apenas criar um conjunto de dados para treinar o modelo com alguns exemplos de respostas que mostram o formato esperado dos dados. Em um caso de uso real, você precisaria criar um conjunto de dados com muito mais exemplos. Também pode ser possível usar um conjunto de dados aberto (para o seu domínio de aplicação), se existir, e reformatá-lo para usar no ajuste fino.

Como estamos focando no `gpt-35-turbo` e queremos uma resposta de turno único (chat completion), podemos criar exemplos usando [este formato sugerido](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) que reflete os requisitos de chat completion da OpenAI. Se você espera conteúdo conversacional de múltiplos turnos, deve usar o [formato de exemplo para múltiplos turnos](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), que inclui um parâmetro `weight` para indicar quais mensagens devem ser usadas (ou não) no processo de ajuste fino.

Vamos usar o formato mais simples de turno único para nosso tutorial aqui. Os dados estão no [formato jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst), com 1 registro por linha, cada um representado como um objeto em formato JSON. O trecho abaixo mostra 2 registros como exemplo – veja [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) para o conjunto completo de exemplos (10 exemplos) que usaremos no nosso tutorial de ajuste fino. **Observação:** Cada registro _deve_ ser definido em uma única linha (não dividido em várias linhas como é comum em um arquivo JSON formatado)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Em um caso de uso real, você vai precisar de um conjunto de exemplos muito maior para obter bons resultados – o equilíbrio será entre a qualidade das respostas e o tempo/custo do ajuste fino. Estamos usando um conjunto pequeno para podermos concluir o ajuste fino rapidamente e ilustrar o processo. Veja [este exemplo do OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) para um tutorial de ajuste fino mais complexo.


### Etapa 1.2 Faça o upload do seu conjunto de dados

Envie os dados usando a API de Arquivos [como descrito aqui](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Lembre-se de que, para executar este código, você precisa ter feito os seguintes passos antes:
 - Instalado o pacote Python `openai` (certifique-se de usar a versão >=0.28.0 para acessar os recursos mais recentes)
 - Definido a variável de ambiente `OPENAI_API_KEY` com sua chave de API da OpenAI
Para saber mais, confira o [Guia de configuração](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) disponibilizado para o curso.

Agora, execute o código para criar um arquivo para upload a partir do seu arquivo JSONL local.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Etapa 2.1: Crie o job de Fine-tuning com o SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Etapa 2.2: Verifique o status do job

Aqui estão algumas coisas que você pode fazer com a API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Lista os últimos n jobs de fine-tuning
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Mostra detalhes de um job de fine-tuning específico
- `client.fine_tuning.jobs.cancel(<job_id>)` - Cancela um job de fine-tuning
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Lista até n eventos do job
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

O primeiro passo do processo é _validar o arquivo de treinamento_ para garantir que os dados estejam no formato correto.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Etapa 2.3: Acompanhe eventos para monitorar o progresso


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Etapa 2.4: Verificar o status no Painel da OpenAI


Você também pode visualizar o status acessando o site da OpenAI e explorando a seção _Fine-tuning_ da plataforma. Lá, você poderá ver o status do trabalho atual e também acompanhar o histórico de execuções anteriores. Nesta captura de tela, é possível ver que a execução anterior falhou, e a segunda tentativa foi bem-sucedida. Para contextualizar, isso aconteceu porque a primeira execução usou um arquivo JSON com registros formatados incorretamente – depois de corrigido, a segunda execução foi concluída com sucesso e o modelo ficou disponível para uso.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.br.png)


Você também pode visualizar as mensagens de status e métricas rolando mais para baixo no painel visual, como mostrado:

| Mensagens | Métricas |
|:---|:---|
| ![Mensagens](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.br.png) |  ![Métricas](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.br.png)|


### Etapa 3.1: Recupere o ID e teste o modelo ajustado no código


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Etapa 3.2: Carregue e teste o modelo ajustado no Playground

Agora você pode testar o modelo ajustado de duas formas. Primeiro, você pode acessar o Playground e usar o menu suspenso de Modelos para selecionar seu modelo ajustado entre as opções disponíveis. A outra opção é usar o botão "Playground" exibido no painel de ajuste fino (veja a captura de tela acima), que abre uma visualização _comparativa_ mostrando lado a lado as versões do modelo base e do modelo ajustado para uma avaliação rápida.

![Status do trabalho de ajuste fino](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.br.png)

Basta preencher o contexto do sistema usado nos seus dados de treinamento e inserir sua pergunta de teste. Você vai perceber que ambos os lados são atualizados com o mesmo contexto e pergunta. Execute a comparação e veja a diferença entre as respostas dos modelos. _Repare como o modelo ajustado gera a resposta no formato que você definiu nos exemplos, enquanto o modelo base apenas segue o prompt do sistema_.

![Status do trabalho de ajuste fino](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.br.png)

Você vai notar que a comparação também mostra a contagem de tokens para cada modelo e o tempo levado para a inferência. **Este exemplo específico é bem simples, serve apenas para mostrar o processo e não reflete um conjunto de dados ou cenário real**. Pode ser que ambos os exemplos exibam o mesmo número de tokens (o contexto do sistema e o prompt do usuário são idênticos), mas o modelo ajustado leva mais tempo para inferir (modelo personalizado).

No mundo real, você não vai usar um exemplo simples como este, mas sim ajustar o modelo com dados reais (por exemplo, catálogo de produtos para atendimento ao cliente), onde a qualidade da resposta será muito mais evidente. Nesse contexto, para obter uma resposta equivalente com o modelo base, será necessário fazer uma engenharia de prompt mais personalizada, o que aumenta o uso de tokens e possivelmente o tempo de processamento para inferência. _Para experimentar, confira os exemplos de ajuste fino no OpenAI Cookbook para começar._



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automáticas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte oficial. Para informações críticas, recomenda-se a tradução profissional feita por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes do uso desta tradução.
